In [1]:
!python -V
import gc as garbage_colector

Python 3.10.4


# <center> GENERADORES</center>
# <center> YIELD vs RETURN</center>

In [2]:
def iterador_generadores(generador):

    while generador:        
        try:
            print(next(generador))
        except StopIteration:
            print('Finalizo iteracion')
            break

def lenguaje():
    yield 'Python'
    yield 'Java'
    yield 'Scala'
               
print(list(lenguaje()))

['Python', 'Java', 'Scala']


In [3]:
iterador_generadores(lenguaje())

Python
Java
Scala
Finalizo iteracion


# <center> GENERADOR USANDO LIST COMPREHENSION

In [4]:
lenguajes = (valor for valor in ['Python', 'Java', 'Scala'])
print(lenguajes)

<generator object <genexpr> at 0x7f26cc398d60>


In [5]:
iterador_generadores(lenguajes)

# elimina la variable creada, recupera el espacio de la variable liberandolo de la memoria con el garbage collector
del lenguajes
garbage_colector.collect(), garbage_colector.get_threshold()

Python
Java
Scala
Finalizo iteracion


(0, (700, 10, 10))

# <center> YIELD FROM</center>

In [6]:
def lista_generador():
    
    yield from 'JorgeCardona'
    
iterador_generadores(lista_generador())

J
o
r
g
e
C
a
r
d
o
n
a
Finalizo iteracion


# <center> **YIELD & YIELD FROM** DENTRO DE OTRO YIELD</center>

In [7]:
def astronomia():
    yield 'Sol'
    yield 'Luna'
    yield 'Estrellas'
    yield 'Galaxias'
    yield 'Planetas'   

def numeros():
    yield 'Uno'
    yield 'Dos'
    yield lenguaje()
    yield from astronomia()
    yield 'Tres'
    yield 'Cuatro'
    yield lista_generador()
    yield 'Cinco'


print(list(numeros()))

['Uno', 'Dos', <generator object lenguaje at 0x7f26cc398f20>, 'Sol', 'Luna', 'Estrellas', 'Galaxias', 'Planetas', 'Tres', 'Cuatro', <generator object lista_generador at 0x7f26cc398f90>, 'Cinco']


# <center> **.\__\___next\__\___()**</center>

In [8]:
def iterador_tareas(lista_funciones):    
    # itera la lista de funciones con generadores
    while lista_funciones:
        
        print(lista_funciones, len(lista_funciones))
        
        try:
            # obtiene la funcion que tiene generadores
            actual = lista_funciones.pop(0)
            print(actual.__next__())                     
        except Exception:
            # para evaluar si es de tipo Generator
            import types
            print(f"{actual} -> {'Generador sin elementos' if isinstance(actual, types.GeneratorType) else 'No es una expresion Generadora'} , es una  {type(actual)}")
            pass
        else:
            lista_funciones.append(actual)
            
        
iterador_tareas([lista_generador(), lenguaje(), astronomia(), numeros()])

[<generator object lista_generador at 0x7f26cc398f90>, <generator object lenguaje at 0x7f26cc398c10>, <generator object astronomia at 0x7f26cc399000>, <generator object numeros at 0x7f26cc3992a0>] 4
J
[<generator object lenguaje at 0x7f26cc398c10>, <generator object astronomia at 0x7f26cc399000>, <generator object numeros at 0x7f26cc3992a0>, <generator object lista_generador at 0x7f26cc398f90>] 4
Python
[<generator object astronomia at 0x7f26cc399000>, <generator object numeros at 0x7f26cc3992a0>, <generator object lista_generador at 0x7f26cc398f90>, <generator object lenguaje at 0x7f26cc398c10>] 4
Sol
[<generator object numeros at 0x7f26cc3992a0>, <generator object lista_generador at 0x7f26cc398f90>, <generator object lenguaje at 0x7f26cc398c10>, <generator object astronomia at 0x7f26cc399000>] 4
Uno
[<generator object lista_generador at 0x7f26cc398f90>, <generator object lenguaje at 0x7f26cc398c10>, <generator object astronomia at 0x7f26cc399000>, <generator object numeros at 0x7f26c

# <center> **ENVIAR VALORES A UN GENERADOR**</center>

In [9]:
def multiplicar_valor():
    while True:
        x,y = yield
        yield x + y
gen = multiplicar_valor()

In [10]:
for i in range(10,20):
    next(gen)
    x = i//2
    y = i
    
    resultado = gen.send([x,y])
    
    print(f'el valor de {x} + {y} es {resultado}')
    
del gen
garbage_colector.collect(), garbage_colector.get_threshold()

el valor de 5 + 10 es 15
el valor de 5 + 11 es 16
el valor de 6 + 12 es 18
el valor de 6 + 13 es 19
el valor de 7 + 14 es 21
el valor de 7 + 15 es 22
el valor de 8 + 16 es 24
el valor de 8 + 17 es 25
el valor de 9 + 18 es 27
el valor de 9 + 19 es 28


(0, (700, 10, 10))

# <center> **ENVIAR VALORES A UN GENERADOR CON CONDICIONALES**</center>

In [11]:
def multiplicar_valor_impar(generador):
    
    while generador:
        # obtiene el valor actual del iterador en el generador
        actual = next(generador)
        print(f'El valor del iterador es {actual}')
        
        # si el valor del generador no es par se le envian parametros
        if actual%2 !=-0:
            
            # parametros a capturar con el metodo send
            x,y = yield
            
            # define las variables a capturar
            if (x and y) is not None:
                yield f'el valor de ({x} x {actual}) + ({y} x {actual} x {2}) es ({x * actual} + {y * actual * 2}) = {x*actual + y*actual*2}'
                print()
        else:
            print(f'El valor {actual} -->, no es un numero impar')
            print()
                
# define el numero maximo de objetos generadores creados
iterador = (i for i in range(10))
gen = multiplicar_valor_impar(iterador)

In [12]:
# rango superior al total de objetos generadores
for i in range(99):
    
    try:
        next(gen)
    except Exception:
        print('Iteracion Finalizada')
        break
    else:
        resultado_parametros = gen.send([i+2,i+5])
        print(resultado_parametros)
        
del gen, iterador
garbage_colector.collect(), garbage_colector.get_threshold()

El valor del iterador es 0
El valor 0 -->, no es un numero impar

El valor del iterador es 1
el valor de (2 x 1) + (5 x 1 x 2) es (2 + 10) = 12

El valor del iterador es 2
El valor 2 -->, no es un numero impar

El valor del iterador es 3
el valor de (3 x 3) + (6 x 3 x 2) es (9 + 36) = 45

El valor del iterador es 4
El valor 4 -->, no es un numero impar

El valor del iterador es 5
el valor de (4 x 5) + (7 x 5 x 2) es (20 + 70) = 90

El valor del iterador es 6
El valor 6 -->, no es un numero impar

El valor del iterador es 7
el valor de (5 x 7) + (8 x 7 x 2) es (35 + 112) = 147

El valor del iterador es 8
El valor 8 -->, no es un numero impar

El valor del iterador es 9
el valor de (6 x 9) + (9 x 9 x 2) es (54 + 162) = 216

Iteracion Finalizada


(0, (700, 10, 10))

# <center> **EXCEPCIONES EN UN GENERADOR**</center>

In [13]:
numeros = (valor for valor in range(10))

def iterador_generadores(generador):

    while generador:        
        try:            
            valor = next(generador)
            print(valor)
            
            # criterio de parada
            if valor > 5:
                generador.throw(ValueError("EL VALOR HA SUPERADO EL LIMITE PERMITIDO"))
                
        except StopIteration:
            print('Finalizo iteracion')
            break
            
iterador_generadores(numeros)

0
1
2
3
4
5
6


ValueError: EL VALOR HA SUPERADO EL LIMITE PERMITIDO

# <center> **DETENER O FINALIZAR UN GENERADOR**</center>

In [ ]:
numeros = (valor for valor in range(10))

def iterador_generadores(generador):

    while generador:        
        try:            
            valor = next(generador)               
        except StopIteration:
            print('Finalizo iteracion')
            break
        else:
            # criterio de parada
            if valor >= 5:
                generador.close()
            print(valor) 

iterador_generadores(numeros)

# <center> **PIPE O CANALIZACIONES CON GENERADORES**</center>

In [14]:
def fibonacci(nums):
    x, y = 0, 1
    for _ in range(nums):
        x, y = y, x+y
        yield x

def cuadrado(nums):
    for num in nums:
        yield num**2

print(list(fibonacci(10)))
print(list(cuadrado(fibonacci(10))))
print(sum(cuadrado(fibonacci(10))))
garbage_colector.collect(), garbage_colector.get_threshold()

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]
[1, 1, 4, 9, 25, 64, 169, 441, 1156, 3025]
4895


(1047, (700, 10, 10))

# <center> **CONSUMO DE MEMORIA EN BYTES DE UNA LISTA VS UN GENERADOR**</center>

In [15]:
import sys
lista = [i * 2 for i in range(10000)]
print(sys.getsizeof(lista))

del lista
garbage_colector.collect(), garbage_colector.get_threshold()

85176


(0, (700, 10, 10))

In [16]:
generador = (i ** 2 for i in range(10000))

while generador: 

    try:
        siguiente = next(generador)
        print(f'tamano del valor generado {sys.getsizeof(siguiente)}, tamano del generador {sys.getsizeof(generador)}, valor {siguiente}')
    except StopIteration:
        print('Finalizo iteracion')
        break
        
del generador
garbage_colector.collect(), garbage_colector.get_threshold()

tamano del valor generado 24, tamano del generador 104, valor 0
tamano del valor generado 28, tamano del generador 104, valor 1
tamano del valor generado 28, tamano del generador 104, valor 4
tamano del valor generado 28, tamano del generador 104, valor 9
tamano del valor generado 28, tamano del generador 104, valor 16
tamano del valor generado 28, tamano del generador 104, valor 25
tamano del valor generado 28, tamano del generador 104, valor 36
tamano del valor generado 28, tamano del generador 104, valor 49
tamano del valor generado 28, tamano del generador 104, valor 64
tamano del valor generado 28, tamano del generador 104, valor 81
tamano del valor generado 28, tamano del generador 104, valor 100
tamano del valor generado 28, tamano del generador 104, valor 121
tamano del valor generado 28, tamano del generador 104, valor 144
tamano del valor generado 28, tamano del generador 104, valor 169
tamano del valor generado 28, tamano del generador 104, valor 196
tamano del valor generad

(0, (700, 10, 10))

# <center> **VELOCIDAD DE PROCESAMIENTO VS CONSUMO DE MEMORIA DE UNA LISTA VS UN GENERADOR**</center>

In [17]:
import cProfile

cProfile.run('sum([i * 2 for i in range(10000)])')

         5 function calls in 0.002 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.002    0.002    0.002    0.002 <string>:1(<listcomp>)
        1    0.000    0.000    0.002    0.002 <string>:1(<module>)
        1    0.000    0.000    0.002    0.002 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.sum}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [18]:
cProfile.run('sum((i * 2 for i in range(10000)))')

         10005 function calls in 0.018 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    10001    0.010    0.000    0.010    0.000 <string>:1(<genexpr>)
        1    0.000    0.000    0.018    0.018 <string>:1(<module>)
        1    0.000    0.000    0.018    0.018 {built-in method builtins.exec}
        1    0.008    0.008    0.018    0.018 {built-in method builtins.sum}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




# <center> iter()</center>

In [19]:
iterador = iter([1,2,3,4,5,6,7,8,9,0])
print(dir(iterador))

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__length_hint__', '__lt__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__']


In [20]:
print(iterador.__next__())
del iterador
garbage_colector.collect(), garbage_colector.get_threshold()

1


(0, (700, 10, 10))

In [21]:
def is_palindrome(num):
    
    r = str(abs(num)) if type(int) else num

    if len(r) < 2:
        return False
    
    elif len(r) %2 == 0:
        size = len(r)//2
        a = r[:size]
        b = r[size:][::-1]
        
        return a==b
        

    else:
        size = len(r)//2
        a = r[:size]
        b = r[size+1:][::-1]
        
        return a==b
        
listado = list()

for i in range(-1000,1000):

    if (is_palindrome(i)):
        listado.append(i)

print(listado)

del listado
garbage_colector.collect(), garbage_colector.get_threshold()

[-999, -989, -979, -969, -959, -949, -939, -929, -919, -909, -898, -888, -878, -868, -858, -848, -838, -828, -818, -808, -797, -787, -777, -767, -757, -747, -737, -727, -717, -707, -696, -686, -676, -666, -656, -646, -636, -626, -616, -606, -595, -585, -575, -565, -555, -545, -535, -525, -515, -505, -494, -484, -474, -464, -454, -444, -434, -424, -414, -404, -393, -383, -373, -363, -353, -343, -333, -323, -313, -303, -292, -282, -272, -262, -252, -242, -232, -222, -212, -202, -191, -181, -171, -161, -151, -141, -131, -121, -111, -101, -99, -88, -77, -66, -55, -44, -33, -22, -11, 11, 22, 33, 44, 55, 66, 77, 88, 99, 101, 111, 121, 131, 141, 151, 161, 171, 181, 191, 202, 212, 222, 232, 242, 252, 262, 272, 282, 292, 303, 313, 323, 333, 343, 353, 363, 373, 383, 393, 404, 414, 424, 434, 444, 454, 464, 474, 484, 494, 505, 515, 525, 535, 545, 555, 565, 575, 585, 595, 606, 616, 626, 636, 646, 656, 666, 676, 686, 696, 707, 717, 727, 737, 747, 757, 767, 777, 787, 797, 808, 818, 828, 838, 848, 858

(0, (700, 10, 10))

# <center> TIPOS DE INSTANCIA</center>

# <center> SINCRONISMO - HILOS - PROCESOS - ASINCRONISMO</center>

<center>
    <img width="50%" lign="center" src="ejecuciones.png">
</center>

# <center> EJECUCION **SINCRONA**</center>

In [22]:
from datetime import datetime
from time import sleep
LIMITE = 11

def dormir_sincrono(tiempo):
    print('corriendo tarea de tiempo',tiempo)
    sleep(tiempo)
    return tiempo

In [23]:
%%time
datos = []

for tiempo_espera in range(1, LIMITE):
    datos.append(dormir_sincrono(tiempo_espera))

print(datos)
print(f'suma de valores de tiempo evaluados {sum(datos)}')

del datos
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo 1
corriendo tarea de tiempo 2
corriendo tarea de tiempo 3
corriendo tarea de tiempo 4
corriendo tarea de tiempo 5
corriendo tarea de tiempo 6
corriendo tarea de tiempo 7
corriendo tarea de tiempo 8
corriendo tarea de tiempo 9
corriendo tarea de tiempo 10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
suma de valores de tiempo evaluados 55
CPU times: user 38.5 ms, sys: 11.4 ms, total: 50 ms
Wall time: 55.1 s


(0, (700, 10, 10))

# <center> EJECUCION **ASINCRONA** Multitarea cooperativa</center>

In [24]:
import asyncio

# para .ipynb, event loop esta listo, usar await
# await main()

# para .py, usar asyncio.run
# asyncio.run(funcion_asincrona())

async def corrutina(tiempo:int) -> int:
    print('corriendo tarea de tiempo',tiempo)
    await asyncio.sleep(tiempo)
    return tiempo

datos = []
resul = []

inicia = datetime.now()

# crea el listado de corrutinas
for tiempo_espera in range(1, LIMITE):
    datos.append(asyncio.gather(corrutina(tiempo_espera)))
    
# recupera los valores de las corrutinas
for corrutina in datos:
    valor = await corrutina
    resul.append(valor[0])

finaliza = datetime.now()
print(f'Wall time: {finaliza.second - inicia.second} s') 
print(resul)

del datos, resul, inicia, finaliza
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo 1
corriendo tarea de tiempo 2
corriendo tarea de tiempo 3
corriendo tarea de tiempo 4
corriendo tarea de tiempo 5
corriendo tarea de tiempo 6
corriendo tarea de tiempo 7
corriendo tarea de tiempo 8
corriendo tarea de tiempo 9
corriendo tarea de tiempo 10
Wall time: 10 s
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


(0, (700, 10, 10))

# <center> EJECUCION **HILO** Subprocesamiento múltiple</center>
## <center> Este módulo construye interfaces de hilado de alto nivel sobre el módulo de más bajo nivel _thread. 

In [25]:
# libreria para procesamiento con hilos
from threading import Thread

# libreria que permite guardar los valores retornados en una cola
from queue import Queue as cola_classica

def almacenar_en_cola(f):
    def wrapper(*args):
        almacenar_resultados.put(f(*args))
    return wrapper

@almacenar_en_cola
def dormir_sincrono(tiempo):
    print('corriendo tarea de tiempo',tiempo)
    sleep(tiempo)
    return tiempo

def obtener_resultado_cola_tarea(cola):
    
    valores = []
    for indice_tarea in range(cola.qsize()):
        valores.append(cola.get(indice_tarea))
    
    return valores

In [26]:
%%time

# instancia de la cola para adicionar los resultados
almacenar_resultados = cola_classica()

# almacena las instancias generadas
listado_instancias = []   
    
# genera la instancias necesarias para ejecutar, las inicializa y las guarda en la lista
for tiempo_espera in range(1, LIMITE):
  
    instancia = Thread(name=f'ejecucion de tarea # {tiempo_espera}', target=dormir_sincrono, args=(tiempo_espera,))

    instancia.start() # Comienza la actividad del hilo. Esto debe llamarse como máximo una vez por objeto de hilo.
    
    listado_instancias.append(instancia)

# recorre la instancias generadass en la lista de instancias y bloque las ejecuciones hasta que termine    
for tipo_instancia in listado_instancias:    
        tipo_instancia.join() # Espera a que salgan los hilos de trabajo. Espera a que el hilo termine. Esto bloquea el hilo llamador hasta que el hilo cuyo método join() es llamado finalice.

# ejecuta la funcion que se encarga de recuperar los resultados de las ejecuciones        
obtener_resultado_cola_tarea(almacenar_resultados)

# elimina las variables creaadas para la ejecucion
del almacenar_resultados, listado_instancias

# muestra la cantidad de objetos que se mantienen despues de la ejecucion  y el espacio ocupado por estos
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo 1
corriendo tarea de tiempo 2
corriendo tarea de tiempo 3
corriendo tarea de tiempo 4
corriendo tarea de tiempo 5
corriendo tarea de tiempo 6
corriendo tarea de tiempo 7
corriendo tarea de tiempo 8
corriendo tarea de tiempo 9
corriendo tarea de tiempo 10
CPU times: user 27.9 ms, sys: 11 ms, total: 38.9 ms
Wall time: 10 s


(0, (700, 10, 10))

## <center> INSTANCIAS CON **LIST COMPREHENSION**

In [27]:
%%time

# instancia de la cola para adicionar los resultados
almacenar_resultados = cola_classica()

# almacena las instancias generadas
listado_instancias = []   

# genera la instancias necesarias para ejecutar, las inicializa y las guarda en la lista
instancias =  [Thread(name=f'ejecucion de tarea # {tiempo_espera}', target=dormir_sincrono, args=(tiempo_espera,)) for tiempo_espera in range(1, LIMITE)]
    
# recorre cada instancia, la inicializa y luego la guarda en la lista de instancias
for instancia in instancias:

    instancia.start() # Comienza la actividad del hilo. Esto debe llamarse como máximo una vez por objeto de hilo.
    
    listado_instancias.append(instancia)

# recorre la instancias generadass en la lista de instancias y bloque las ejecuciones hasta que termine    
for tipo_instancia in listado_instancias:    
        tipo_instancia.join() # Espera a que salgan los hilos de trabajo. Espera a que el hilo termine. Esto bloquea el hilo llamador hasta que el hilo cuyo método join() es llamado finalice.

# ejecuta la funcion que se encarga de recuperar los resultados de las ejecuciones        
obtener_resultado_cola_tarea(almacenar_resultados)

# elimina las variables creaadas para la ejecucion
del almacenar_resultados, listado_instancias

# muestra la cantidad de objetos que se mantienen despues de la ejecucion  y el espacio ocupado por estos
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo 1
corriendo tarea de tiempo 2
corriendo tarea de tiempo 3
corriendo tarea de tiempo 4
corriendo tarea de tiempo 5
corriendo tarea de tiempo 6
corriendo tarea de tiempo 7
corriendo tarea de tiempo 8
corriendo tarea de tiempo 9
corriendo tarea de tiempo 10
CPU times: user 46.5 ms, sys: 201 µs, total: 46.7 ms
Wall time: 10 s


(0, (700, 10, 10))

# <center> EJECUCION **PROCESO MULTIPLE**</center>
## <center> Este módulo permite crear procesos (spawning) utilizando una API similar al módulo threading. ofrece concurrencia tanto local como remota. permite aprovechar al máximo múltiples procesadores en una máquina determinada. 

In [28]:
# libreria que permite usar la computacion paralela
from multiprocessing import Process

# libreria que permite guardar los valores retornados en una cola
from multiprocessing import Queue as cola_proceso

def almacenar_en_cola_proceso(f):
    def wrapper(*args):
        almacenar_resultados_proceso.put(f(*args))
    return wrapper

@almacenar_en_cola_proceso
def dormir_sincrono(tiempo):
    print('corriendo tarea de tiempo',tiempo)
    sleep(tiempo)
    return tiempo

def obtener_resultado_cola_tarea_while(cola):
    
    valores = []
    while not cola.empty():
        result = cola.get()
        valores.append(result)    
    return valores

In [29]:
%%time

# instancia de la cola para adicionar los resultados
almacenar_resultados_proceso = cola_proceso()

# almacena las instancias generadas
listado_instancias = []

for tiempo_espera in range(1, LIMITE):
  
    instancia = Process(name=f'ejecucion de tarea # {tiempo_espera}', target=dormir_sincrono, args=(tiempo_espera,))
    instancia.start() # Comienza la actividad del proceso. Esto debe llamarse como máximo una vez por objeto de proceso.  
    listado_instancias.append(instancia)

for tipo_instancia in listado_instancias:        
        tipo_instancia.join() # Espera a que salgan los procesos de trabajo. Se debe llamar close() o terminate() antes de usar join().

obtener_resultado_cola_tarea_while(almacenar_resultados_proceso)

del almacenar_resultados_proceso, listado_instancias
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo corriendo tarea de tiempo1 
corriendo tarea de tiempo2
 3corriendo tarea de tiempocorriendo tarea de tiempo  
corriendo tarea de tiempo5 46


corriendo tarea de tiempocorriendo tarea de tiempocorriendo tarea de tiempo  corriendo tarea de tiempo 7109

 
8
CPU times: user 65.5 ms, sys: 74.1 ms, total: 140 ms
Wall time: 10.1 s


(0, (700, 10, 10))

# <center> TOTAL PROCESADORES

In [30]:
import psutil
total_cpu = psutil.cpu_count()
mitad_procesamiento = total_cpu//2
total_cpu, mitad_procesamiento

(4, 2)

# <center> EJECUCION **THREAD POOL DE MULTIPROCESO**</center>

In [31]:
def dormir_sincrono_multiprocessing(tiempo):
    print('corriendo tarea de tiempo ',tiempo)
    sleep(tiempo)
    return tiempo

# <center> **PROCESADORES POR DEFECTO**</center>

In [32]:
# libreria quenpermite usar multihilos con la libreria multiprocessing
from multiprocessing.pool import ThreadPool

In [33]:
%%time

pool = ThreadPool()
multihilo = pool.map(dormir_sincrono_multiprocessing, range(1, LIMITE))
pool.close()
pool.join()

print(multihilo)

del pool, multihilo
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo  1
corriendo tarea de tiempo  2
corriendo tarea de tiempo  3
corriendo tarea de tiempo  4
corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
CPU times: user 54.1 ms, sys: 1.06 ms, total: 55.2 ms
Wall time: 18.1 s


(0, (700, 10, 10))

# <center> **PROCESADORES ASIGNADOS** processes</center>

In [34]:
%%time

with ThreadPool(processes=mitad_procesamiento) as pool_de_hilos:
    multihilo = pool_de_hilos.map(dormir_sincrono_multiprocessing, range(1, LIMITE))

print(multihilo)

del pool_de_hilos, multihilo
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo  1
corriendo tarea de tiempo  3
corriendo tarea de tiempo  2
corriendo tarea de tiempo corriendo tarea de tiempo  5
 4
corriendo tarea de tiempo  7
corriendo tarea de tiempo  6
corriendo tarea de tiempo  9
corriendo tarea de tiempo  8
corriendo tarea de tiempo  10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
CPU times: user 50.3 ms, sys: 0 ns, total: 50.3 ms
Wall time: 33.1 s


(0, (700, 10, 10))

# <center> EJECUCION **POOL DE MULTIPROCESO**</center>

# <center> **PROCESADORES POR DEFECTO**</center>

In [35]:
# libreria quenpermite usar multiprocesos con la libreria multiprocessing
from multiprocessing.pool import Pool

In [36]:
%%time

pool = Pool()
multiproceso = pool.map(dormir_sincrono_multiprocessing, range(1, LIMITE))
pool.close() # Impide que se envíen más tareas a la piscina (pool). Una vez que se hayan completado todas las tareas, se cerrarán los procesos de trabajo.
pool.join() # Espera a que salgan los procesos de trabajo. Se debe llamar close() o terminate() antes de usar join().

print(multiproceso)

del pool, multiproceso
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo corriendo tarea de tiempo   corriendo tarea de tiempo  1corriendo tarea de tiempo 2
4 
3

corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
CPU times: user 76.3 ms, sys: 42.9 ms, total: 119 ms
Wall time: 18.1 s


(0, (700, 10, 10))

## <center> **PROCESOS ADMINISTRADOR DE CONTEXTO**</center>

In [37]:
%%time
with Pool() as pool_de_procesos:
    multiproceso = pool_de_procesos.map(dormir_sincrono_multiprocessing, range(1, LIMITE))

print(multiproceso)

del pool_de_procesos, multiproceso
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo corriendo tarea de tiempo corriendo tarea de tiempo   4 2corriendo tarea de tiempo  

13

corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
CPU times: user 82.2 ms, sys: 34.8 ms, total: 117 ms
Wall time: 18.1 s


(0, (700, 10, 10))

# <center> **PROCESADORES ASIGNADOS** processes</center>

In [38]:
%%time
with Pool(processes=mitad_procesamiento) as pool_de_procesos:
    multiproceso = pool_de_procesos.map(dormir_sincrono_multiprocessing, range(1, LIMITE))
    
print(multiproceso)

del pool_de_procesos, multiproceso
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo corriendo tarea de tiempo   13

corriendo tarea de tiempo  2
corriendo tarea de tiempo  4
corriendo tarea de tiempo  5
corriendo tarea de tiempo  7
corriendo tarea de tiempo  6
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
CPU times: user 60.3 ms, sys: 45.6 ms, total: 106 ms
Wall time: 33.1 s


(0, (700, 10, 10))

# <center> EJECUCION **THREAD POOL EXECUTOR DE CONCURRENT.FUTURES**</center>
## <center> Este módulo provee una interfaz de alto nivel para ejecutar invocables de forma **asincrónica**. Usando **ThreadPoolExecutor, o ProcessPoolExecutor**. Ambos implementan la misma interfaz definida por la **clase abstracta Executor** compatible con asyncio.

In [39]:
def dormir_concurrent_futures(tiempo):
    print('corriendo tarea de tiempo ',tiempo)
    sleep(tiempo)
    return tiempo

# <center> **PROCESADORES POR DEFECTO**</center>

In [40]:
# importa librerias para trabajar concurrencia
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed

In [41]:
%%time

executor = ThreadPoolExecutor()

futures = []
resultado = []
for tiempo in range(1, LIMITE):
    future = executor.submit(dormir_concurrent_futures, tiempo)
    futures.append(future)

    for future in as_completed(futures):
        resultado.append(future.result())

executor.shutdown()
print(resultado)

del executor, futures, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo  1
corriendo tarea de tiempo  2
corriendo tarea de tiempo  3
corriendo tarea de tiempo  4
corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
[1, 1, 2, 2, 1, 3, 3, 2, 1, 4, 3, 2, 4, 1, 5, 4, 1, 2, 5, 3, 6, 4, 1, 2, 5, 3, 6, 7, 7, 4, 1, 2, 5, 3, 6, 8, 7, 4, 1, 2, 8, 5, 3, 6, 9, 7, 4, 1, 2, 8, 5, 3, 9, 6, 10]
CPU times: user 39.1 ms, sys: 11.9 ms, total: 51 ms
Wall time: 55.1 s


(0, (700, 10, 10))

# <center> **EL ADMINISTRADOR DE CONTEXTO** NO GARANTIZA EL ORDEN</center>

In [42]:
%%time

futures = []
resultado = []

with ThreadPoolExecutor() as executor:
    for tiempo in range(1, LIMITE):
        future = executor.submit(dormir_concurrent_futures, tiempo)
        futures.append(future)    

    for future in as_completed(futures):
        resultado.append(future.result())
    
print(resultado)

del executor, futures, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo  1
corriendo tarea de tiempo  2
corriendo tarea de tiempo  3
corriendo tarea de tiempo  4
corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
CPU times: user 52.5 ms, sys: 0 ns, total: 52.5 ms
Wall time: 12.1 s


(0, (700, 10, 10))

# <center> **PROCESADORES ASIGNADOS ADMINISTRADOR DE CONTEXTO** max_workers</center>

In [43]:
%%time

futures = []
resultado = []

with ThreadPoolExecutor(max_workers=mitad_procesamiento) as executor:
    for tiempo in range(1, LIMITE):
        future = executor.submit(dormir_concurrent_futures, tiempo)
        futures.append(future)    

    for future in as_completed(futures):
        resultado.append(future.result())
    
print(resultado)

del executor, futures, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo  1
corriendo tarea de tiempo  2
corriendo tarea de tiempo  3
corriendo tarea de tiempo  4
corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
CPU times: user 41.1 ms, sys: 8.86 ms, total: 49.9 ms
Wall time: 30 s


(0, (700, 10, 10))

# <center> **PROCESADORES ASIGNADOS MAP** MANTIENE EL ORDEN DE LA INFORMACION</center>

In [44]:
%%time

with ThreadPoolExecutor(max_workers=mitad_procesamiento) as executor:
    futures = executor.map(dormir_concurrent_futures, range(1, LIMITE))
list(futures) 

del executor, futures
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo  1
corriendo tarea de tiempo  2
corriendo tarea de tiempo  3
corriendo tarea de tiempo  4
corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
CPU times: user 44.4 ms, sys: 1.85 ms, total: 46.2 ms
Wall time: 30.1 s


(0, (700, 10, 10))

# <center> EJECUCION **PROCESS POOL EXECUTOR DE CONCURRENT.FUTURES**</center>

In [45]:
# importa librerias para trabajar concurrencia
from concurrent.futures import ProcessPoolExecutor

# <center> **PROCESADORES POR DEFECTO**</center>

In [46]:
%%time

executor = ProcessPoolExecutor()

futures = []
resultado = []
for tiempo in range(1, LIMITE):
    future = executor.submit(dormir_concurrent_futures, tiempo) # submit -> Programa el invocable, fn, para que se ejecute como fn(*args, **kwargs) y devuelve un objeto Future que representa la ejecución del invocable.
    futures.append(future)

# as_completed -> espera que se complete cada llamado de future para poder mostrar su resultado.
# usar as_completed -> cuando se una el metodo submit para correr una funcion
for future in as_completed(futures):
    resultado.append(future.result()) # .result() obtiene el resultado de ejecutor

executor.shutdown() # Indica al ejecutor que debe liberar todos los recursos que está utilizando cuando los futuros actualmente pendientes de ejecución finalicen.

print(resultado)

del executor, futures, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo  1corriendo tarea de tiempo 
 2corriendo tarea de tiempo 
corriendo tarea de tiempo   4
3
corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
CPU times: user 90 ms, sys: 36.7 ms, total: 127 ms
Wall time: 18.1 s


(0, (700, 10, 10))

# <center> **EL ADMINISTRADOR DE CONTEXTO** NO GARANTIZA EL ORDEN</center>

In [47]:
%%time

futures = []
resultado = []

with ProcessPoolExecutor() as executor:
    for tiempo in range(1, LIMITE):
        future = executor.submit(dormir_concurrent_futures, tiempo)
        futures.append(future)    

    for future in as_completed(futures):
        resultado.append(future.result())
    
print(resultado)

del executor, futures, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo  1corriendo tarea de tiempo  2corriendo tarea de tiempo 
corriendo tarea de tiempo  4

 3
corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
CPU times: user 91 ms, sys: 30.1 ms, total: 121 ms
Wall time: 18.1 s


(0, (700, 10, 10))

# <center> **PROCESADORES POR DEFECTO USANDO MAP**</center>

In [48]:
%%time

with ProcessPoolExecutor() as executor:
    futures = executor.map(dormir_concurrent_futures, range(1, LIMITE))
list(futures)

del executor, futures
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo  corriendo tarea de tiempo 1
 corriendo tarea de tiempo 2corriendo tarea de tiempo  
4
 3
corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
CPU times: user 42.9 ms, sys: 79.4 ms, total: 122 ms
Wall time: 18.1 s


(0, (700, 10, 10))

# <center> **PROCESADORES ASIGNADOS** max_workers</center>

In [49]:
%%time

futures = []
resultado = []
with ProcessPoolExecutor(max_workers=2) as executor:    
    futures = executor.map(dormir_concurrent_futures, range(1, LIMITE))
list(futures)  

del executor, futures, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo  corriendo tarea de tiempo 1
 2
corriendo tarea de tiempo  3
corriendo tarea de tiempo  4
corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
CPU times: user 74.8 ms, sys: 37.2 ms, total: 112 ms
Wall time: 30.1 s


(0, (700, 10, 10))

# *****************************************************************************************************************************************************************************************************************************

# <center> HILOS I/O VS PROCESOS I/O</center>

In [50]:
cantidad = 100_000

def lectura_io(value):
    with open('demofile.txt', 'rb') as f:
        return f.read(value * 100)

## <center> HILOS I/O</center>

### <center> LIMITAR LA CANTIDAD DE TAREAS PARA NO DESBORDAR LA MEMORIA POR ALMACENAMIENTO DE RESULTADOS EN LA COLA</center>

In [51]:
%%time
almacenar_resultados = cola_classica()
listado_instancias = [] 

@almacenar_en_cola
def lectura_io_hilo(value):
    with open('demofile.txt', 'rb') as f:
        return f.read(value * 100)
    
for valor_lectura in range(1, cantidad//2):
  
    instancia = Thread(name=f'ejecucion de tarea # {valor_lectura}', target=lectura_io_hilo, args=(valor_lectura,))
    instancia.start()    
    listado_instancias.append(instancia)

    
for tipo_instancia in listado_instancias:    
        tipo_instancia.join()
        
resultado = obtener_resultado_cola_tarea(almacenar_resultados)
print(resultado[:100])

del almacenar_resultados, listado_instancias, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

[b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge

(0, (700, 10, 10))

### <center> CAPACIDAD DE PROCESAR TODAS LAS TAREAS SIN DESBORDAR LA MEMORIA</center>

In [52]:
%%time

with ThreadPool() as multi_hilos:        
    resultado = multi_hilos.map(lectura_io, range(cantidad))
    print(len(resultado))
    
print(list(resultado)[:100])

del multi_hilos, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

100000
[b'', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronis

(0, (700, 10, 10))

In [53]:
%%time

with ThreadPoolExecutor() as multi_hilos:        
    resultado = multi_hilos.map(lectura_io, range(cantidad))
    print(len(list(resultado)))
    
print(list(resultado))

del multi_hilos, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

100000
[]
CPU times: user 23.2 s, sys: 58.9 s, total: 1min 22s
Wall time: 36.9 s


(0, (700, 10, 10))

## <center> PROCESOS I/O</center>

In [54]:
%%time
almacenar_resultados_proceso = cola_proceso()
listado_instancias = [] 

@almacenar_en_cola_proceso
def lectura_io_proceso(value):
    with open('demofile.txt', 'rb') as f:
        return f.read(value * 100)
    
for valor_lectura in range(1, cantidad//200):
  
    instancia = Process(name=f'ejecucion de tarea # {valor_lectura}', target=lectura_io_proceso, args=(valor_lectura,))
    instancia.start()    
    listado_instancias.append(instancia)

    
for tipo_instancia in listado_instancias:    
        tipo_instancia.join()
        
resultado = obtener_resultado_cola_tarea_while(almacenar_resultados_proceso)
print(resultado[:100])

del almacenar_resultados_proceso, listado_instancias, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

[b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge

(0, (700, 10, 10))

In [55]:
%%time
with Pool() as multiprocessing:
    resultado = multiprocessing.map(lectura_io, range(cantidad))
    print(len(list(resultado)))
    
print(list(resultado)[:100])

del multiprocessing, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

100000
[b'', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronis

(0, (700, 10, 10))

In [56]:
%%time
with ProcessPoolExecutor() as concurrent_futures:
    resultado = concurrent_futures.map(lectura_io, range(cantidad))
    
print(list(resultado)[:100])

del concurrent_futures, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

[b'', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'

(0, (700, 10, 10))

# <center> HILOS CPU VS PROCESOS CPU</center>

In [57]:
import math

inicio = 1_099_726_899_294_999
fin    = 1_099_726_899_299_999

PRIMES = range(inicio, fin) 

def is_prime(n):
    if n < 2 or n % 2 == 0:
        return False
    if n == 2:
        return True
    
    sqrt_n = int(math.floor(math.sqrt(n)))
    
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True

print(f'total iteraciones {fin - inicio}')

total iteraciones 5000


## <center> SINCRONO CPU</center>

In [58]:
%%time

print(list(map(is_prime, PRIMES)))

[False, False, False, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False

## <center> HILOS CPU</center>

In [59]:
%%time
almacenar_resultados = cola_classica()
listado_instancias = [] 

@almacenar_en_cola
def is_prime_hilo(n):
    if n < 2 or n % 2 == 0:
        return False
    if n == 2:
        return True
    
    sqrt_n = int(math.floor(math.sqrt(n)))
    
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True
    
for valor_lectura in PRIMES:
  
    instancia = Thread(name=f'ejecucion de tarea # {valor_lectura}', target=is_prime_hilo, args=(valor_lectura,))
    instancia.start()    
    listado_instancias.append(instancia)

    
for tipo_instancia in listado_instancias:    
        tipo_instancia.join()
        
resultado = obtener_resultado_cola_tarea(almacenar_resultados)
print(resultado[:100])

del almacenar_resultados, listado_instancias, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
CPU times: user 5min 22s, sys: 0 ns, total: 5min 22s
Wall time: 5min 8s


(0, (700, 10, 10))

In [60]:
%%time
resultado = []

with ThreadPool() as executor:
    for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):       
        resultado.append((number, prime))

print('cantidad de valores procesados', len(resultado))

del executor, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

cantidad de valores procesados 5000
CPU times: user 5min 2s, sys: 0 ns, total: 5min 2s
Wall time: 4min 55s


(0, (700, 10, 10))

In [61]:
%%time
resultado = []

with ThreadPoolExecutor() as executor:
    for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):       
        resultado.append((number, prime))

print('cantidad de valores procesados', len(resultado))
        
del executor, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

cantidad de valores procesados 5000
CPU times: user 5min 31s, sys: 0 ns, total: 5min 31s
Wall time: 5min 16s


(0, (700, 10, 10))

## <center> PROCESOS CPU</center>

In [62]:
%%time
almacenar_resultados_proceso = cola_proceso()
listado_instancias = [] 

@almacenar_en_cola_proceso
def is_prime_proceso(n):
    if n < 2 or n % 2 == 0:
        return False
    if n == 2:
        return True
    
    sqrt_n = int(math.floor(math.sqrt(n)))
    
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True
    
for valor_lectura in PRIMES:
  
    instancia = Process(name=f'ejecucion de tarea # {valor_lectura}', target=is_prime_proceso, args=(valor_lectura,))
    instancia.start()    
    listado_instancias.append(instancia)

    
for tipo_instancia in listado_instancias:    
        tipo_instancia.join()

print('cantidad de valores procesados', len(obtener_resultado_cola_tarea_while(almacenar_resultados_proceso)))

del almacenar_resultados_proceso, listado_instancias
garbage_colector.collect(), garbage_colector.get_threshold()

cantidad de valores procesados 5000
CPU times: user 38.2 s, sys: 0 ns, total: 38.2 s
Wall time: 3min 18s


(0, (700, 10, 10))

In [63]:
%%time

resultado = []
with Pool() as executor:
    for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):       
        resultado.append((number, prime))

print('cantidad de valores procesados', len(resultado))

del executor, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

cantidad de valores procesados 5000
CPU times: user 178 ms, sys: 0 ns, total: 178 ms
Wall time: 2min 52s


(0, (700, 10, 10))

In [64]:
%%time

resultado = []
with ProcessPoolExecutor() as executor:
    for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):       
        resultado.append((number, prime))

print('cantidad de valores procesados', len(resultado))
        
del executor, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

cantidad de valores procesados 5000
CPU times: user 2.04 s, sys: 0 ns, total: 2.04 s
Wall time: 2min 53s


(0, (700, 10, 10))

## <center> NUMBA CPU</center>

In [65]:
#pip install numba # Numba es el paquete compilador, esto depende de llvmlite.
#pip install llvmlite # llvmlite es un paquete de enlace ligero para las API de LLVM, depende de LLVM, LLVM es el marco del compilador JIT para producir código ejecutable a partir de varias entradas.
!pip show numba

Name: numba
Version: 0.55.1
Summary: compiling Python code using LLVM
Home-page: https://numba.pydata.org
Author: 
Author-email: 
License: BSD
Location: /usr/local/lib/python3.10/site-packages
Requires: llvmlite, numpy, setuptools
Required-by: 


In [66]:
from numba import jit

@jit(nopython=True)
def is_prime_numba(n):
    if n < 2 or n % 2 == 0:
        return False
    if n == 2:
        return True
    
    sqrt_n = int(math.floor(math.sqrt(n)))
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True

In [67]:
%%time
print(f'total iteraciones {fin - inicio}')
print(list(map(is_prime_numba, PRIMES)))

garbage_colector.collect(), garbage_colector.get_threshold()

total iteraciones 5000
[False, False, False, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, Tr

(3644, (700, 10, 10))

In [68]:
%%time

inicio = 1_099_726_899_249_999
fin    = 1_099_726_899_299_999

PRIMES = range(inicio, fin) 

print(f'total iteraciones {fin - inicio}')
print(list(map(is_prime_numba, PRIMES)))
garbage_colector.collect(), garbage_colector.get_threshold()

total iteraciones 50000
[False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, Fals

(0, (700, 10, 10))

# *****************************************************************************************************************************************************************************************************************************

# <center> EJECUCION 1800 TAREAS DE 2 SEGUNDOS, TOMARIA 1 HORA EN EJECUCION SINCRONA</center>

In [69]:
import random
limite_tareas = 1800

## <center> ASINCRONO</center>

In [70]:
async def corrutina(tiempo:int) -> int:    
    await asyncio.sleep(tiempo)
    return random.randint(0, limite_tareas)

inicia = datetime.now()
lista = [corrutina(2) for i in range(limite_tareas)]

resultado = await asyncio.gather(*lista)

finaliza = datetime.now()
print(len(resultado))
print(f'Wall time: {finaliza.second - inicia.second} s')
print(resultado)

del lista, resultado, inicia, finaliza
garbage_colector.collect(), garbage_colector.get_threshold()

1800
Wall time: 2 s
[932, 779, 1117, 1155, 360, 755, 892, 1124, 1455, 1460, 1420, 1145, 1514, 1676, 1745, 1251, 1659, 1665, 1429, 648, 784, 1302, 1578, 1496, 587, 1137, 536, 1638, 1721, 546, 1438, 610, 1473, 470, 1406, 960, 757, 216, 1705, 1518, 186, 1013, 116, 1083, 1244, 503, 792, 1762, 218, 170, 1778, 232, 1276, 1056, 608, 1363, 1014, 1526, 416, 732, 1766, 1666, 917, 466, 1142, 502, 1754, 1399, 1215, 1597, 320, 1362, 1630, 1162, 803, 824, 1068, 321, 36, 1337, 61, 960, 1211, 1496, 1043, 657, 747, 1205, 879, 1791, 1719, 1772, 1030, 1338, 1179, 1468, 227, 1485, 726, 309, 805, 1582, 1342, 1429, 1026, 26, 898, 1032, 260, 777, 1059, 1616, 1113, 673, 808, 713, 1023, 1029, 190, 1702, 682, 1515, 416, 307, 310, 1187, 578, 1568, 1381, 1171, 240, 993, 771, 1442, 124, 1670, 1434, 420, 297, 772, 1517, 874, 1011, 1087, 1005, 872, 522, 1402, 281, 1587, 1778, 157, 204, 1287, 1004, 1441, 1157, 150, 210, 1561, 666, 207, 1497, 824, 1049, 1596, 1766, 1187, 1181, 161, 875, 317, 781, 951, 1773, 1289, 1565

(0, (700, 10, 10))

## <center> THREAD</center>

In [71]:
%%time

listado_instancias = []
# instancia de la cola para adicionar los resultados
almacenar_resultados = cola_classica()


@almacenar_en_cola
def dormir_sincrono(tiempo:int) -> int:    
    sleep(tiempo)
    return random.randint(0,limite_tareas)

    
for tiempo_espera in range(limite_tareas):
  
    instancia = Thread(name=f'ejecucion de tarea # {tiempo_espera}', target=dormir_sincrono, args=(2,))

    instancia.start()
    
    listado_instancias.append(instancia)

    
for tipo_instancia in listado_instancias:    
        tipo_instancia.join()

resultado = obtener_resultado_cola_tarea(almacenar_resultados)
print(len(resultado))
print(resultado)

del listado_instancias, almacenar_resultados, instancia, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

1800
[1658, 765, 1549, 1436, 1714, 904, 1319, 1469, 402, 1310, 916, 1524, 255, 1623, 1731, 976, 1432, 1470, 1650, 1654, 256, 1631, 1575, 382, 1366, 1190, 789, 1322, 505, 855, 1092, 1543, 887, 53, 555, 1310, 243, 1298, 674, 1467, 530, 1623, 1362, 879, 16, 71, 22, 1424, 1454, 1338, 1193, 739, 600, 593, 1495, 1476, 1063, 194, 828, 795, 94, 1738, 1533, 206, 1574, 1783, 559, 1189, 554, 777, 870, 1649, 546, 585, 727, 1599, 1793, 1789, 1144, 883, 1311, 536, 141, 57, 220, 34, 1567, 1118, 1442, 1527, 139, 1097, 1012, 597, 923, 260, 323, 674, 785, 1643, 712, 469, 1684, 1048, 1127, 643, 182, 1239, 996, 35, 391, 1212, 385, 1196, 1038, 33, 1692, 1693, 986, 1415, 242, 56, 1543, 545, 1561, 1421, 0, 658, 1118, 974, 972, 1222, 626, 798, 1655, 311, 418, 95, 228, 1630, 316, 1481, 474, 899, 1400, 1592, 558, 765, 326, 753, 651, 1491, 714, 586, 1093, 1281, 579, 933, 338, 272, 1748, 257, 793, 284, 954, 1483, 1381, 225, 1436, 1238, 553, 1075, 356, 759, 621, 1514, 1292, 764, 352, 764, 1346, 887, 1578, 1378, 72

(0, (700, 10, 10))

## <center> PROCESS</center>

In [72]:
%%time

# instancia de la cola para adicionar los resultados
listado_instancias = []

# instancia de la cola para adicionar los resultados
almacenar_resultados_proceso = cola_proceso()

@almacenar_en_cola_proceso
def dormir_sincrono(tiempo:int) -> int:    
    sleep(tiempo)
    return random.randint(0,limite_tareas)
        

for tiempo_espera in range(limite_tareas):
  
    instancia = Process(name=f'ejecucion de tarea # {tiempo_espera}', target=dormir_sincrono, args=(2,))
    instancia.start()
    
    listado_instancias.append(instancia)

    
for tipo_instancia in listado_instancias:    
        tipo_instancia.join()

resultado = obtener_resultado_cola_tarea_while(almacenar_resultados_proceso)
print(len(resultado))
print(resultado)

del listado_instancias, almacenar_resultados_proceso, instancia, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

1800
[18, 233, 858, 281, 217, 1768, 1180, 846, 1464, 543, 543, 28, 1295, 1669, 352, 1233, 276, 1583, 954, 1783, 1574, 1682, 16, 672, 188, 1033, 1187, 1539, 1534, 51, 1412, 1045, 1549, 40, 408, 1323, 1572, 25, 1299, 149, 996, 637, 1512, 1510, 948, 1444, 62, 699, 307, 1007, 1263, 915, 528, 1278, 1536, 1754, 63, 234, 1329, 1519, 1455, 1227, 420, 1050, 477, 812, 1612, 348, 1105, 1053, 1324, 1040, 32, 594, 1061, 1792, 523, 964, 1268, 1510, 1443, 1638, 1295, 78, 774, 1482, 32, 1077, 971, 1114, 183, 779, 609, 1294, 1363, 73, 979, 1419, 1268, 181, 1383, 755, 137, 587, 792, 665, 56, 447, 1172, 105, 972, 648, 1106, 125, 1188, 362, 237, 853, 1337, 91, 124, 1051, 64, 830, 560, 427, 1475, 521, 1445, 650, 232, 970, 1641, 654, 1672, 128, 1786, 1799, 149, 24, 101, 1170, 1075, 1244, 240, 742, 846, 1548, 709, 944, 1635, 1676, 456, 243, 724, 1502, 1348, 826, 300, 1195, 28, 662, 694, 858, 379, 413, 517, 328, 590, 1677, 1401, 265, 789, 39, 1744, 1189, 1569, 1473, 998, 1288, 1427, 1256, 310, 1444, 619, 1651

(0, (700, 10, 10))

## <center> **POOL PROCESS DE MULTIPROCESO**</center>

In [73]:
%%time

def dormir_sincrono_multiprocessing(contador:int) -> int:    
    sleep(2)
    return random.randint(0,limite_tareas)

with Pool() as multiprocesing:
    pool = multiprocesing.map(dormir_sincrono_multiprocessing, range(limite_tareas//3))
print(list(pool))

del multiprocesing, pool
garbage_colector.collect(), garbage_colector.get_threshold()

[448, 43, 706, 1109, 501, 652, 1050, 372, 1279, 67, 1107, 146, 937, 1647, 540, 1726, 1717, 1260, 261, 1045, 1654, 1514, 1276, 132, 1073, 942, 581, 159, 701, 1015, 37, 279, 1770, 1421, 42, 1205, 976, 1764, 1252, 349, 43, 1037, 955, 1581, 770, 542, 132, 1519, 756, 352, 913, 1636, 520, 92, 598, 1745, 1710, 704, 853, 969, 474, 1544, 872, 1231, 995, 1719, 741, 556, 800, 729, 1734, 101, 369, 78, 1728, 1698, 1267, 933, 604, 1800, 1316, 754, 863, 1703, 1280, 1557, 129, 1508, 1744, 42, 462, 1728, 315, 307, 34, 1299, 1781, 856, 649, 1361, 313, 368, 1608, 1638, 727, 1444, 1671, 1600, 97, 847, 997, 933, 823, 1209, 688, 786, 1172, 59, 764, 1351, 1350, 484, 1650, 1470, 243, 1141, 233, 1573, 877, 300, 218, 455, 48, 149, 1217, 480, 828, 1396, 1750, 1609, 1682, 20, 1746, 1436, 1355, 742, 779, 1229, 732, 1245, 606, 404, 23, 1175, 862, 40, 311, 1150, 815, 1711, 1495, 458, 825, 26, 1290, 1736, 1317, 136, 814, 62, 1229, 258, 496, 197, 664, 1280, 843, 250, 995, 1568, 1443, 706, 1185, 1193, 323, 1419, 405, 5

(0, (700, 10, 10))

## <center> **THREAD POOL DE MULTIPROCESO**</center>

In [74]:
%%time

with ThreadPool() as multiprocesing:
    pool = multiprocesing.map(dormir_sincrono_multiprocessing, range(limite_tareas//3))
print(list(pool))

del multiprocesing, pool
garbage_colector.collect(), garbage_colector.get_threshold()

[1631, 1196, 1754, 1155, 474, 586, 1692, 249, 405, 56, 1039, 1585, 1583, 1768, 907, 1749, 556, 476, 1708, 1585, 1175, 858, 203, 1644, 862, 222, 1156, 1755, 522, 591, 280, 598, 681, 1075, 557, 1454, 946, 1472, 684, 224, 1074, 671, 1203, 458, 1458, 1789, 54, 809, 1521, 396, 1053, 1589, 293, 386, 1088, 1101, 773, 1730, 752, 1080, 320, 30, 950, 518, 1397, 279, 768, 1172, 330, 290, 1028, 1477, 81, 1248, 149, 1608, 464, 339, 1172, 155, 1185, 191, 1329, 67, 652, 747, 1541, 718, 216, 1053, 249, 897, 1366, 97, 1635, 624, 663, 631, 1553, 842, 251, 1016, 1800, 1698, 1619, 632, 109, 1360, 398, 1666, 125, 587, 89, 309, 1609, 1246, 1453, 706, 906, 387, 977, 563, 77, 1527, 1612, 516, 1322, 1576, 511, 545, 629, 1387, 1214, 1055, 266, 1186, 162, 1754, 159, 1113, 1189, 685, 322, 672, 695, 894, 1205, 796, 1467, 1180, 1254, 691, 1742, 1761, 91, 1649, 1282, 1435, 1261, 234, 1798, 1291, 759, 1673, 657, 1483, 650, 15, 397, 636, 144, 1370, 765, 54, 1563, 742, 1292, 1201, 445, 871, 413, 192, 1018, 724, 850, 77

(0, (700, 10, 10))

## <center> **PROCESS POOL EXECUTOR DE CONCURRENT.FUTURES**</center>

In [75]:
%%time

with ProcessPoolExecutor() as executor:
    pool = executor.map(dormir_sincrono_multiprocessing, range(limite_tareas//3))
print(list(pool))

del executor, pool
garbage_colector.collect(), garbage_colector.get_threshold()

[507, 101, 422, 450, 1570, 1756, 1715, 338, 19, 205, 955, 1308, 599, 534, 1435, 1401, 1176, 1415, 1599, 1553, 1671, 819, 271, 1460, 1575, 1288, 255, 594, 1600, 355, 248, 1142, 784, 1361, 1786, 463, 551, 1467, 1615, 868, 253, 995, 880, 734, 43, 649, 157, 1225, 303, 452, 145, 141, 701, 472, 1648, 823, 833, 1324, 1688, 214, 887, 530, 296, 1617, 71, 1235, 754, 1319, 994, 413, 652, 51, 1657, 169, 537, 1314, 1502, 716, 1289, 921, 1642, 619, 671, 304, 1484, 583, 634, 947, 1008, 712, 60, 1181, 360, 1680, 166, 1317, 154, 1746, 448, 214, 928, 136, 1241, 744, 335, 2, 1566, 217, 1136, 1448, 1242, 1560, 922, 618, 1023, 1009, 1668, 147, 921, 1526, 1346, 392, 1479, 618, 467, 1630, 1708, 981, 975, 364, 640, 1633, 847, 1412, 278, 395, 1037, 550, 1720, 327, 1523, 1735, 1526, 927, 814, 1248, 1254, 483, 1731, 237, 1597, 1294, 148, 963, 70, 1723, 870, 1514, 261, 1152, 577, 1632, 415, 584, 646, 1545, 1573, 1762, 977, 1056, 823, 1064, 1715, 293, 1565, 1418, 1487, 1616, 1696, 462, 996, 1186, 788, 720, 1792, 6

(0, (700, 10, 10))

## <center> **THREAD POOL EXECUTOR DE CONCURRENT.FUTURES**</center>

In [76]:
%%time

with ThreadPoolExecutor() as executor:
    pool = executor.map(dormir_sincrono_multiprocessing, range(limite_tareas//3))
print(list(pool))

del executor, pool
garbage_colector.collect(), garbage_colector.get_threshold()

[883, 64, 225, 446, 84, 1263, 711, 1313, 1748, 855, 991, 1508, 898, 53, 1096, 1455, 970, 473, 199, 371, 181, 792, 169, 562, 374, 96, 1566, 470, 1473, 754, 1545, 882, 1688, 184, 1691, 459, 441, 89, 133, 1771, 1786, 1346, 1090, 1272, 1389, 1274, 1646, 496, 314, 312, 914, 1616, 303, 949, 1087, 792, 742, 381, 1146, 1529, 1485, 871, 1098, 372, 371, 1674, 77, 178, 672, 1153, 275, 819, 490, 132, 1665, 872, 678, 710, 1284, 478, 304, 1264, 96, 966, 74, 112, 1120, 686, 80, 1113, 1430, 1391, 1063, 1327, 344, 1349, 1380, 586, 64, 1772, 1020, 1610, 1634, 635, 913, 909, 975, 563, 1279, 1320, 837, 548, 118, 1367, 1542, 1479, 93, 1715, 465, 774, 85, 1274, 24, 1045, 275, 1430, 1441, 505, 309, 695, 360, 886, 361, 946, 1554, 812, 377, 1549, 719, 100, 1179, 466, 272, 1037, 1237, 978, 1082, 1716, 1077, 1644, 839, 1714, 498, 1112, 214, 176, 19, 398, 1786, 1715, 251, 473, 822, 1166, 202, 733, 1496, 832, 1383, 568, 88, 1128, 1443, 1576, 299, 232, 1360, 1157, 347, 1181, 1117, 798, 1655, 334, 1643, 1786, 1059, 

(0, (700, 10, 10))

## <center> **COMPARTIR DATOS ENTRE HILOS Y PROCESOS**</center>

## <center> Compartir Datos Globales entre Hilos. 

In [77]:
%%time

compartir = 0
limite_hilos = 4
limite_iterador = 25

def memoria_compartida():
    
    global compartir    
    
    compartir_local  = compartir    
    
    for i in range(limite_iterador):
        compartir_local += 1
        
    sleep(0.2)
    
    for i in range(limite_iterador):
        compartir_local += 1
       
    compartir  = compartir_local
    
    print(f'Valor parcial {compartir}')

hilos = []

for hilo_numero in range(limite_hilos):
    
    hilo = Thread(target=memoria_compartida)
    hilo.start() 
    
    hilos.append(hilo)
    
for instancia_hilo in hilos:
    
    instancia_hilo.join()
    

print(f'Valor final {compartir}')

del compartir, limite_hilos, limite_iterador
garbage_colector.collect(), garbage_colector.get_threshold()

Valor parcial 50
Valor parcial 50
Valor parcial 50
Valor parcial 50
Valor final 50
CPU times: user 42.7 ms, sys: 10.7 ms, total: 53.4 ms
Wall time: 251 ms


(0, (700, 10, 10))

## <center> Definiendo la seccion critica. 

In [78]:
%%time

from threading import Lock
# instancia que bloquea hasta que finalice la seccion critica
seccion_critica = Lock()

compartir = 0
limite_hilos = 4
limite_iterador = 25

def memoria_compartida():
    
    global compartir    
    
    # debe estar ANTES de asignar el valor de la variable, que es el INICIO de la seccion critica
    seccion_critica.acquire()
    
    compartir_local  = compartir    
    
    for i in range(limite_iterador):
        compartir_local += 1
        
    sleep(0.2)
    
    for i in range(limite_iterador):
        compartir_local += 1
       
    compartir  = compartir_local
    
    # debe estar DESPUES de asignar el valor de la variable, que es el FINAL de la seccion critica
    seccion_critica.release()
    
    print(f'Valor parcial {compartir}')

hilos = []

for hilo_numero in range(limite_hilos):
    
    hilo = Thread(target=memoria_compartida)
    hilo.start()
    
    hilos.append(hilo)
    
for instancia_hilo in hilos:
    
    instancia_hilo.join()
    

print(f'Valor final {compartir}')

del seccion_critica, compartir, limite_hilos, limite_iterador
garbage_colector.collect(), garbage_colector.get_threshold()

Valor parcial 50
Valor parcial 100
Valor parcial 150
Valor parcial 200
Valor final 200
CPU times: user 56.2 ms, sys: 0 ns, total: 56.2 ms
Wall time: 851 ms


(0, (700, 10, 10))

## <center> Compartir Datos entre Procesos. 

In [79]:
%%time

compartir = 0
limite_procesos = 4
limite_iterador = 25
    
def datos_compartidos():
    
    global compartir
    
    compartir_local  = compartir
    
    for i in range(limite_iterador):
        compartir_local += 1
        
    sleep(0.2)
    
    for i in range(limite_iterador):
        compartir_local += 1
     
    compartir  = compartir_local
    print(f'Valor parcial {compartir}')

procesos = []

for hilo_numero in range(limite_procesos):
    
    proceso = Process(target=datos_compartidos)
    proceso.start()
    
    procesos.append(proceso)
    
for instancia_proceso in procesos:
    
    instancia_proceso.join()
    

print(f'Valor final {compartir}')

del compartir, limite_procesos, limite_iterador
garbage_colector.collect(), garbage_colector.get_threshold()

Valor parcial 50
Valor parcial 50
Valor parcial 50
Valor parcial 50
Valor final 0
CPU times: user 98.6 ms, sys: 74.1 ms, total: 173 ms
Wall time: 363 ms


(0, (700, 10, 10))

## <center> Sin definir la seccion critica. 

In [80]:
%%time
from multiprocessing import Value

# instancia que permite compartir los valores, definiendo el tipo de dato int
compartir = Value('i',0)

limite_procesos = 4
limite_iterador = 25

def datos_compartidos(): 
    
    global compartir
    
    for i in range(limite_iterador):
        compartir.value += 1
        
    sleep(0.2)
    
    for i in range(limite_iterador):
        compartir.value += 1
    
    print(f'Valor parcial {compartir.value}')

procesos = []

for hilo_numero in range(limite_procesos):
    
    proceso = Process(target=datos_compartidos)
    proceso.start()
    
    procesos.append(proceso)
    
for instancia_proceso in procesos:
    
    instancia_proceso.join()
    

print(f'Valor final {compartir.value}')

del compartir, limite_procesos, limite_iterador
garbage_colector.collect(), garbage_colector.get_threshold()

Valor parcial 125
Valor parcial 150
Valor parcial 175
Valor parcial 200
Valor final 200
CPU times: user 92.3 ms, sys: 32.7 ms, total: 125 ms
Wall time: 323 ms


(0, (700, 10, 10))

## <center> Definiendo la seccion critica. 

In [81]:
%%time
from multiprocessing import Lock

# instancia que bloquea hasta que finalice la seccion critica
seccion_critica = Lock()

# instancia que permite compartir los valores
compartir = Value('i',0)

limite_procesos = 4
limite_iterador = 25

def datos_compartidos():
    
    global compartir
    
    # debe estar ANTES de asignar el valor de la variable, que es el INICIO de la seccion critica
    seccion_critica.acquire() 
    
    for i in range(limite_iterador):
        compartir.value += 1
        
    sleep(0.2)
    
    for i in range(limite_iterador):
        compartir.value += 1
    
    # debe estar DESPUES de asignar el valor de la variable, que es el FINAL de la seccion critica
    seccion_critica.release()
    
    print(f'Valor parcial {compartir.value}')

procesos = []

for hilo_numero in range(limite_procesos):
    
    proceso = Process(target=datos_compartidos)
    proceso.start()
    
    procesos.append(proceso)
    
for instancia_proceso in procesos:
    
    instancia_proceso.join()
    

print(f'Valor final {compartir.value}')

del seccion_critica, compartir, limite_procesos, limite_iterador
garbage_colector.collect(), garbage_colector.get_threshold()

Valor parcial 50
Valor parcial 100
Valor parcial 150
Valor parcial 200
Valor final 200
CPU times: user 71.8 ms, sys: 62.9 ms, total: 135 ms
Wall time: 897 ms


(0, (700, 10, 10))

## <center> Compartir Datos entre THREAD POOL. 
### <center> SIN SECCION CRITICA. 

In [82]:
# instancia que permite compartir los valores
compartir = Value('i',0)
seccion_critica = Lock()

def datos_compartidos(incremento): 
    
    compartir.value += incremento
    sleep(0.2)    
    print(f'Valor parcial {compartir.value}')
    
    return compartir.value
 
with ThreadPool() as pool_de_hilos:
    multihilo = pool_de_hilos.map(datos_compartidos, range(1, LIMITE))

print(multihilo)
del multihilo, pool_de_hilos, compartir, seccion_critica
garbage_colector.collect(), garbage_colector.get_threshold()

Valor parcial 10Valor parcial 10
Valor parcial 15
Valor parcial 21

Valor parcial 36
Valor parcial 45
Valor parcial 55
Valor parcial 55
Valor parcial 55
Valor parcial 55
[21, 15, 28, 10, 36, 55, 55, 45, 55, 55]


(0, (700, 10, 10))

### <center> CON SECCION CRITICA. 

In [83]:
# instancia que permite compartir los valores
compartir = Value('i',0)
seccion_critica = Lock()

def datos_compartidos(incremento): 
    
    seccion_critica.acquire()
    compartir.value += incremento
    sleep(0.2)
    
    print(f'Valor parcial {compartir.value}') # acceder al valor antes de retornarlo puede interferir en el valor final retornado
    seccion_critica.release()   
    
    
    return compartir.value
    
with ThreadPool() as pool_de_hilos:
    multihilo = pool_de_hilos.map(datos_compartidos, range(1, LIMITE))

print(multihilo)
del multihilo, pool_de_hilos, compartir, seccion_critica
garbage_colector.collect(), garbage_colector.get_threshold()

Valor parcial 1
Valor parcial 6
Valor parcial 9
Valor parcial 13
Valor parcial 15
Valor parcial 21
Valor parcial 28
Valor parcial 36
Valor parcial 45
Valor parcial 55
[1, 15, 9, 13, 6, 21, 28, 36, 45, 55]


(0, (700, 10, 10))

## <center> Compartir Datos entre POOL de Procesos. 
## <center> Sin seccion critica. 

In [84]:
%%time
# instancia que permite compartir los valores
compartir = Value('i',0)
seccion_critica = Lock()

def datos_compartidos(incremento):
    
    global compartir
    
    compartir.value += incremento
    sleep(0.2)
    
    print(f'Valor parcial {compartir.value}')
    
    return compartir.value
    
with Pool() as pool_de_procesos:
    multiproceso = pool_de_procesos.map(datos_compartidos, range(1, LIMITE))
    
print(multiproceso)

del multiproceso, pool_de_procesos, compartir, seccion_critica
garbage_colector.collect(), garbage_colector.get_threshold()

Valor parcial 10Valor parcial 10Valor parcial 10
Valor parcial 10


Valor parcial 36

Valor parcial 36Valor parcial 36
Valor parcial 45
Valor parcial 55
Valor parcial 55
[21, 28, 15, 10, 36, 45, 55, 55, 55, 55]
CPU times: user 87.8 ms, sys: 83.8 ms, total: 172 ms
Wall time: 753 ms


(0, (700, 10, 10))

## <center> Definiendo la seccion critica. 

In [85]:
# instancia que permite compartir los valores
compartir = Value('i',0)
seccion_critica = Lock()

def datos_compartidos(incremento):
    
    global compartir
    
    seccion_critica.acquire()
    
    compartir.value += incremento
    sleep(0.2)
    print(f'Valor parcial {compartir.value}')
    seccion_critica.release()  
      
    return compartir.value
    
    
with Pool() as pool_de_procesos:
    multiproceso = pool_de_procesos.map(datos_compartidos, range(1, LIMITE))
    
print(multiproceso)
del multiproceso, pool_de_procesos, compartir, seccion_critica
garbage_colector.collect(), garbage_colector.get_threshold()

Valor parcial 1
Valor parcial 3
Valor parcial 6
Valor parcial 10
Valor parcial 15
Valor parcial 21
Valor parcial 28
Valor parcial 36
Valor parcial 45
Valor parcial 55
[1, 3, 6, 10, 15, 21, 28, 36, 45, 55]


(0, (700, 10, 10))

## <center> Definiendo la seccion critica. **SIN PRINT**, OBTIENE LOS VALORES DE RETORNO CORRECTAMENTE, POR NO ACCEDER A ELLOS

In [86]:
# instancia que permite compartir los valores
compartir = Value('i',0)
seccion_critica = Lock()

def datos_compartidos(incremento):
    
    global compartir
    
    seccion_critica.acquire()
    
    compartir.value += incremento
    sleep(0.2)
    seccion_critica.release()  
      
    return compartir.value
    
    
with Pool() as pool_de_procesos:
    multiproceso = pool_de_procesos.map(datos_compartidos, range(1, LIMITE))
    
print(multiproceso)
del multiproceso, pool_de_procesos, compartir, seccion_critica
garbage_colector.collect(), garbage_colector.get_threshold()

[1, 3, 6, 10, 15, 21, 28, 36, 45, 55]


(0, (700, 10, 10))

# <center> POOL DE PROCESOS E HILOS OTROS METODOS</center>
## <center>Ofrece un medio conveniente de paralelizar la ejecución de una función a través de múltiples valores de entrada, distribuyendo los datos de entrada a través de procesos (paralelismo de datos).</center>

In [87]:
from multiprocessing.pool import Pool

def calcular_cuadrado(x):
    return x*x

### <center> APPLY - PROCESOS</center>

In [88]:
%%time
pool = Pool()
pool_apply = pool.apply(calcular_cuadrado, [3])
pool.close() # Prevents any more tasks from being submitted to the pool. Once all the tasks have been completed the worker processes will exit.
pool.join() # Wait for the worker processes to exit. One must call close() or terminate() before using join().
pool.close() # Close the Process object, releasing all resources associated with it.
pool_apply

print(pool_apply)
del pool_apply, pool
garbage_colector.collect(), garbage_colector.get_threshold()

9
CPU times: user 62 ms, sys: 63 ms, total: 125 ms
Wall time: 117 ms


(0, (700, 10, 10))

### <center> APPLY - HILOS</center>

In [89]:
%%time
pool = ThreadPool()
pool_apply = pool.apply(calcular_cuadrado, [4])
pool.close() # Prevents any more tasks from being submitted to the pool. Once all the tasks have been completed the worker processes will exit.
pool.join() # Wait for the worker processes to exit. One must call close() or terminate() before using join().
pool.close() # Close the Process object, releasing all resources associated with it.
pool_apply

print(pool_apply)
del pool_apply, pool
garbage_colector.collect(), garbage_colector.get_threshold()

16
CPU times: user 71.6 ms, sys: 9.4 ms, total: 81 ms
Wall time: 78.1 ms


(0, (700, 10, 10))

### <center> APPLY_ASYNC</center>

In [90]:
%%time
with Pool() as pool:
    values = pool.apply_async(calcular_cuadrado, [5])

print(values.get())
del pool, values
garbage_colector.collect(), garbage_colector.get_threshold()

25
CPU times: user 64.8 ms, sys: 54.5 ms, total: 119 ms
Wall time: 119 ms


(0, (700, 10, 10))

### <center> MAP_ASYNC</center>

In [91]:
%%time
with Pool() as pool:
    values = pool.map_async(calcular_cuadrado, [6])

print(values.get())
del pool, values
garbage_colector.collect(), garbage_colector.get_threshold()

[36]
CPU times: user 71.3 ms, sys: 31.7 ms, total: 103 ms
Wall time: 104 ms


(0, (700, 10, 10))

### <center> MAP</center>

In [92]:
%%time
with Pool() as pool:
    values = pool.map(calcular_cuadrado, range(0,20,5))

print(values)
del pool, values
garbage_colector.collect(), garbage_colector.get_threshold()

[0, 25, 100, 225]
CPU times: user 52.2 ms, sys: 53.3 ms, total: 106 ms
Wall time: 110 ms


(0, (700, 10, 10))

### <center> IMAP</center>

In [93]:
%%time
with Pool() as pool:
    values = pool.imap(calcular_cuadrado, range(0,20,4))

print(values)
del pool, values
garbage_colector.collect(), garbage_colector.get_threshold()

CPU times: user 53.1 ms, sys: 53.4 ms, total: 107 ms
Wall time: 110 ms


(152, (700, 10, 10))

### <center> IMAP_UNORDERED</center>

In [94]:
%%time
with Pool() as pool:
    values = pool.imap_unordered(calcular_cuadrado, range(0,20,2))

print(values)
del pool, values
garbage_colector.collect(), garbage_colector.get_threshold()

CPU times: user 50.8 ms, sys: 63.9 ms, total: 115 ms
Wall time: 113 ms


(152, (700, 10, 10))

### <center> STARMAP</center>

In [95]:
%%time
with Pool() as pool:
    values = pool.starmap(calcular_cuadrado, [[1],[2],[3],[4],[5]])

print(values)
del pool, values
garbage_colector.collect(), garbage_colector.get_threshold()

[1, 4, 9, 16, 25]
CPU times: user 56.6 ms, sys: 52.8 ms, total: 109 ms
Wall time: 112 ms


(0, (700, 10, 10))

### <center> STARMAP_ASYNC</center>

In [96]:
%%time
with Pool() as pool:
    values = pool.starmap_async(calcular_cuadrado, [[1],[2],[3],[4],[5]])

print(values)
del pool, values
garbage_colector.collect(), garbage_colector.get_threshold()

CPU times: user 60.2 ms, sys: 53.1 ms, total: 113 ms
Wall time: 116 ms


(153, (700, 10, 10))

# <center> OTRAS **FUNCIONES ASINCRONAS**</center>

In [97]:
async def Primero(limite=5):
    print("Inicio funcion Primero")
    await asyncio.sleep(2)
    print("Final funcion Primero")
    
    return [random.randrange(-100, 0) for x in range(limite)]

async def Ultimo(limite=5):
    print("Comenzar funcion ultimo")
    await asyncio.sleep(2)
    print("Terminar funcion ultimo")
    
    return [random.randrange(0, 100) for x in range(limite)]

# <center> **GATHER**</center>
## <center> **RECIBE UNA LISTA DE TAREAS Y RETORNA UNA LISTA DE RESULTADOS**</center>
## <center> **TIENE OPCIONES ESPECÍFICAS PARA EL MANEJO DE ERRORES Y CANCELACIONES.**</center>

In [98]:
# El siguiente ejemplo muestra cómo esperar a que se completen varias tareas asincrónicas.

from asyncio import gather

async def iniciar_gather():
    
    tareas = [Primero(), Ultimo(), Primero(), Ultimo(), Primero(), Ultimo(), Primero()]
    
    return await gather(*tareas)
    
await iniciar_gather()

Inicio funcion Primero
Comenzar funcion ultimo
Inicio funcion Primero
Comenzar funcion ultimo
Inicio funcion Primero
Comenzar funcion ultimo
Inicio funcion Primero
Final funcion Primero
Terminar funcion ultimo
Final funcion Primero
Terminar funcion ultimo
Final funcion Primero
Terminar funcion ultimo
Final funcion Primero


[[-99, -51, -24, -43, -77],
 [68, 47, 99, 46, 58],
 [-85, -65, -29, -97, -60],
 [72, 15, 96, 20, 20],
 [-85, -8, -7, -44, -89],
 [16, 29, 15, 34, 21],
 [-11, -61, -57, -12, -28]]

# <center> **WAIT_FOR**</center>
## <center> **PERMITE DEFINIR EL MAXIMO TIEMPO DE ESPERA DE UNA TAREA**</center>

In [99]:
# El siguiente ejemplo demuestra cómo podemos utilizar un tiempo de espera para evitar esperar indefinida a que finalice una tarea asincrónica.

from asyncio import wait_for

async def iniciar_wait_for():
    try:
        return await wait_for(Primero(), timeout=1)
    except asyncio.TimeoutError:
        print("la ejecucion supero el tiempo de espera!")
        

await iniciar_wait_for()

Inicio funcion Primero
la ejecucion supero el tiempo de espera!


# <center> **AS_COMPLETED**</center>
## <center> **ES SIMILIAR A GATHER, PERO RETORNA FUTUROS, LOS RESULTADOS SON RETORNADOS EN EL ORDEN QUE ESTAN LISTOS**</center>

In [100]:
# El siguiente ejemplo demuestra cómo as_complete, completará la primera tarea, seguida de la siguiente más rápida y la siguiente hasta que se completen todas las tareas.

from asyncio import as_completed

async def iniciar_as_completed():
    
    tareas = [Primero(), Ultimo(), Primero(), Ultimo(), Primero(), Ultimo(), Primero()]
    counter = 0
    
    for future in as_completed(tareas):
        n = "la tarea mas rapida" if counter == 0 else "la siguiente tarea mas rapida"
        counter += 1
        result = await future
        print(f"{n} obtuvo el resultado: {result}")

await iniciar_as_completed()

Inicio funcion Primero
Comenzar funcion ultimo
Comenzar funcion ultimo
Inicio funcion Primero
Inicio funcion Primero
Inicio funcion Primero
Comenzar funcion ultimo
Final funcion Primero
Terminar funcion ultimo
Terminar funcion ultimo
Final funcion Primero
Final funcion Primero
Final funcion Primero
Terminar funcion ultimo
la tarea mas rapida obtuvo el resultado: [-89, -22, -11, -49, -100]
la siguiente tarea mas rapida obtuvo el resultado: [88, 77, 93, 8, 62]
la siguiente tarea mas rapida obtuvo el resultado: [22, 88, 34, 14, 33]
la siguiente tarea mas rapida obtuvo el resultado: [-5, -97, -6, -18, -35]
la siguiente tarea mas rapida obtuvo el resultado: [-70, -31, -21, -86, -98]
la siguiente tarea mas rapida obtuvo el resultado: [-3, -12, -73, -87, -20]
la siguiente tarea mas rapida obtuvo el resultado: [53, 40, 81, 27, 79]


# <center> **CREATE_TASK**</center>

In [101]:
# El siguiente ejemplo demuestra cómo convertir una rutina en una tarea y programarla en el bucle de eventos.

from asyncio import create_task

async def iniciar_create_task():
    
    tarea = create_task(Primero())
    print(tarea)
    
    await asyncio.sleep(2)
    print("MAS PROCESOS!")
    
    await asyncio.sleep(3)
    print(sum(tarea))
    
    return tarea

await iniciar_create_task()

<Task pending name='Task-1835' coro=<Primero() running at /tmp/ipykernel_27722/4294725927.py:1>>
Inicio funcion Primero
MAS PROCESOS!
Final funcion Primero
0


<Task finished name='Task-1835' coro=<Primero() done, defined at /tmp/ipykernel_27722/4294725927.py:1> result=[-61, -59, -1, -73, -20]>

# <center> **POOLS**</center>
# <center> **GET_RUNNING_LOOP**</center>

## <center> **ASYNCIO Event Loop - MULTI HILO**</center>

In [102]:
# importa la libtreria
from asyncio import get_running_loop
# importa librerias para trabajar concurrencia
from concurrent.futures import ThreadPoolExecutor

def blocking_io(value = 23, otros = 10):
    # File operations (such as logging) can block the
    # event loop: run them in a thread pool.
    with open("/dev/urandom", "rb") as f:
        return f.read(value + otros)
    
async def asyncio_multi_hilo():
    
    loop = get_running_loop()
    pool = ThreadPoolExecutor()
    
    # tipo de pool, funcion, parametros
    return await loop.run_in_executor(pool, blocking_io, 50, 70)

await asyncio_multi_hilo()

b"gKW\xb8\xe9\xcf\xedz\x8c\x8b\x08\x13:\xef\x8azh\x95,\x97\x91\xf4\xe3\xf2\xc9\xd35|\xf0\xcf&\x11&\xbf\x15#E[\xb6%\x91b%z\x10j\xd4c\xd9\xf1\x9a\xae)\xd2\xbdM'&\x04q\x83\x87\xfa%@\x08\x85\xc3\xd6%F\xa1\xb4?=4\xaf\x9f\xe4\x85\x17J\x17u\xff\xe3\xccd\x18\x86\xab\xfe\xcd\xf5\xd7\x9f8\xed\xb1@\x9f\xe6\xf6\xcf\xa3r/\xa1\\1&j\xbf\xf3\x89)Z\xfeM\xa8"